In [1]:
%%html
<script>
(function() {
  // Create the toggle button
  const rtlButton = document.createElement("button");
  rtlButton.textContent = "Toggle LTR";
  rtlButton.id = "top-rtl-toggle";
  rtlButton.style.marginLeft = "8px";
  rtlButton.style.padding = "4px 10px";
  rtlButton.style.fontSize = "14px";
  rtlButton.style.cursor = "pointer";

  // State
  var rtlActive = false;

  // Styling function
  var applyStyleToEditor = (editor) => {
    if (!editor) return;
    var direction = getComputedStyle(editor).getPropertyValue('direction')=='rtl' ? 'ltr' : 'rtl';
    var text_align = getComputedStyle(editor).getPropertyValue('text-align')=='right' ? 'left' : 'right';
    editor.style.setProperty('direction', direction, 'important');
    editor.style.setProperty('text-align', text_align, 'important');
  };

  // Toggle logic
  rtlButton.onclick = () => {
    rtlActive = !rtlActive;
    rtlButton.textContent = rtlActive ? "Toggle LTR" : "Toggle RTL";
    document.querySelectorAll('.jp-MarkdownCell .jp-InputArea-editor').forEach(applyStyleToEditor);
    document.querySelectorAll('.jp-RenderedHTMLCommon code, .jp-RenderedHTMLCommon code span').forEach(applyStyleToEditor);
    document.querySelectorAll('jp-RenderedHTMLCommon, .jp-RenderedHTMLCommon *').forEach(applyStyleToEditor);
  };

  // Watch for focus into editing Markdown cells
  // document.addEventListener('focusin', (event) => {
  //   const editor = event.target.closest('.jp-MarkdownCell .jp-InputArea-editor');
  //    if (editor) applyStyleToEditor(editor);
  // });

  // Insert into top toolbar if not already present
  var insertIntoToolbar = () => {
    const toolbar = document.querySelector('.jp-NotebookPanel-toolbar');
    if (toolbar && !document.getElementById("top-rtl-toggle")) {
      toolbar.appendChild(rtlButton);
    } else {
      // Try again in a moment if toolbar isn't ready yet
      setTimeout(insertIntoToolbar, 300);
    }
  };

  insertIntoToolbar();
})();
</script>

In [2]:
%%html
<!-- <style>
  table {display: inline-block}
</style> -->

# שבוע 7 - Numpy

## ספריות בפייתון
**ספרייה (Library)** היא אוסף מודולים של קוד שנכתב מראש כדי לבצע משימות נפוצות: חישוב מספרי, עיבוד נתונים, גרפיקה, תקשורת, למידת מכונה ועוד. במקום להמציא את הגלגל בכל פרויקט, משתמשים בספריות אמינות שנבדקו על־ידי משתמשים רבים.

- **התקנה**: מוסיפים את הספרייה לסביבה שלכם באמצעות מנהל חבילות (כגון `pip`) כך שהקוד שלה יהיה זמין לפרויקט.  
- **טעינה**: בתוך הקוד “מייבאים” את הספרייה או חלקים ממנה באמצעות פקודת `import`, ולאחר מכן קוראים לפונקציות ולמחלקות שהיא מספקת.  
- **שימוש**: משלבים את הקריאות לספרייה יחד עם הקוד שלכם, לפי התיעוד הרשמי ודוגמאות שימוש. ברוב המקרים, העבודה דומה לקריאה לכל פונקציה אחרת בפייתון—רק שהמימוש מגיע “מבחוץ”.

בפייתון יש שתי צורות נפוצות לייבוא:
1) **ייבוא הספרייה כולה** (לעיתים עם כינוי/alias),  
2) **ייבוא סלקטיבי** של אובייקטים או תת־מודולים מתוכה.

ברוב המקרים, ייבוא עם כינוי שומר על קריאות, מונע התנגשויות שמות, ומבהיר מאין הגיע כל אובייקט. ייבוא סלקטיבי מתאים כשאתם משתמשים במספר קטן של פונקציות שוב ושוב, או כשאתם רוצים להדגיש תלות בתת־מודול ספציפי. 
שימו לב: בספריות כמו NumPy, ייבוא סלקטיבי לרוב **לא** חוסך זמן טעינה או זיכרון בצורה משמעותית, ולעיתים אף מגביר סיכון להתנגשויות שמות.

#### ייבוא ספרייה שלמה עם כינוי:
```python
import numpy as np
```

####  ייבוא סלקטיבי של אובייקטים/תת־מודולים:
```python
from numpy import array, mean #object
from numpy.linalg import norm #sub-module
```

```{note}
נייבא ספרייה שלמה כאשר:
- משתמשים במגוון רחב של יכולות מהחבילה.  
- לשמירה על קריאות עקבית עם כינויים נפוצים (`np`, `pd`, `plt`).  
- להפחתת סיכון להתנגשויות שמות בין פונקציות בעלות אותו שם מספריות שונות.  

נייבא חלקים מספרייה כאשר:
- משתמשים במספר קטן ומוגדר של פונקציות/קלאסים בתדירות גבוהה, ורוצים תחביר קצר יותר.  
- כדי להדגיש תלות בתת־מודול מסוים (למשל `numpy.linalg` לאלגברה ליניארית).  
```

`````{admonition} עקרונות עבודה מומלצים
:class :tip 
- **לייבא רק מה שצריך**: שומר על קוד קריא וממוקד.  
- **הסתמכות על תיעוד**: קראו תיעוד רשמי וקווי הנחיה—זה מקצר עקומת למידה ומונע תקלות.  
`````


## NumPy

NumPy היא ספריית הליבה לחישוב מדעי בפייתון.
היא מאפשרת לעבוד עם אובייקטים מסוג מערך רב־ממדי המאפשרים חישובים מהירים ומספקת כלי עזר לעבודה עם מערכים הללו.

ל**NumPy** חשיבות יתרה בפיסיקה מאחר שהיא מספקת:
- **חישוב מהיר בהיקפים גדולים**: ביצוע פעולות על אוספי נתונים שלמים בפקודה אחת במקום לולאות — קריטי לסימולציות, אינטגרציה נומרית ופתרון בעיות רשת/סריג.
- **אלגברה ליניארית וניתוח ספקטרלי**: כלים מובנים לפתרון מערכות, מציאת ערכים ווקטורים עצמיים וניתוח תדרים — בסיס לבעיות תנודות, שדות ופיזיקה קוונטית.
- **סימולציות הסתברותיות**: דגימה אקראית יעילה ונוחה לביצוע ניסויי מונטה־קרלו, הוספת רעש מדוד והערכת אי־ודאות במדידות.
- **דיוק נומרי אמין**: התאמת רמת הדיוק והעבודה עם מספרים ממשיים ומרוכבים מאפשרות ייצוג נאמן של גדלים פיזיקליים רגישים.
- **יעילות בזיכרון ובמעבד**: ניסוח חישובים כך שיתבצעו בצמוד לנתונים, מצמצם העתקות מיותרות ומאיץ ריצות.
- **עבודה טבעית עם שדות ורשתות**: כתיבה שנשענת על פעולות וקטוריות/מטריציוניות מקבילה ישירות לנוסחאות, ולכן מקלה על אימות תאורטי.
- **קריאות ושחזור**: קוד תמציתי, שקל לבדוק, לשחזר ולשתף — מרכיב חיוני בעבודה מחקרית.
- **זרימה חלקה לניתוח מלא**: משתלבת היטב עם סביבת הכלים המדעיים של פייתון לניהול נתונים, פתרון נומרי והדמיה — מאיסוף ועד מסקנות.


### Numpy's Arrays (מערכים)

מערך של NumPy מכיל ערכים מאותו טיפוס, שמאונדקס באמצעות מספרים טבעיים.
מספר הממדים נקרא דרגת המערך (rank) וה-shape הוא גודל המערך בכל ממד.

ראשית,מייבאים את הספרייה NumPy ומקצרים את השם שלה ל־np, כדי שיהיה נוח להשתמש בה בהמשך.

In [3]:
import numpy as np

נבנה מערך חד־ממדי (Rank 1), מתוך רשימת פייתון [1, 2, 3].
המערך הזה דומה לרשימה רגילה, אבל מאפשר ביצועים טובים יותר ופעולות מתמטיות ישירות על כל האיברים.

In [4]:
a = np.array([1, 2, 3])   # Create a rank 1 array
print(type(a))            

<class 'numpy.ndarray'>


בודקים את סוג המשתנה `a`. רואים שהוא מסוג `numpy.ndarray` — זוהי המחלקה המרכזית של NumPy לייצוג מערכים.

In [5]:
print(a[0], a[1], a[2])   

1 2 3


גישה לאיברים במערך מתבצעת בעזרת אינדקסים בסוגריים מרובעים.
האינדקס מתחיל מ־0 (כמו בפייתון רגילה), ולכן `a[0]` הוא האיבר הראשון (1), `a[1]` הוא השני (2), וכן הלאה.

In [6]:
a[0] = 5                  # Change an element of the array
print(a)                  

[5 2 3]


מערך של NumPy הינו בר־שינוי (mutable).
בקטע הקוד הקודם מחליפים את הערך של האיבר הראשון מ־1 ל־5, והמערך כולו הופך להיות `[5, 2, 3]`.

In [7]:
b = np.array([[1,2,3],[4,5,6]])    # Create a rank 2 array


בקטע הקוד הקודם יוצרים מערך דו־ממדי (Rank 2), מתוך רשימה מקוננת.
הרשימה הראשונה `[1,2,3]` הופכת לשורה ראשונה, והרשימה `[4,5,6]` הופכת לשורה שנייה.
כלומר b מייצג טבלה (מטריצה) עם 2 שורות ו־3 עמודות.

In [8]:
print(b.shape)                     

(2, 3)


כעת הצורה (`shape`) היא `(2,3)`, כלומר שתי שורות ושלושה עמודות.

In [9]:
print(b[0, 0], b[0, 1], b[1, 0])  


1 2 4


גישה לאיברים במערך דו־ממדי נעשית ע״י ציון שורה ועמודה:
- `b[0,0]` הוא התא בשורה הראשונה, עמודה ראשונה → הערך 1.
- `b[0,1]` הוא התא בשורה הראשונה, עמודה שנייה → הערך 2.
- `b[1,0]` הוא התא בשורה השנייה, עמודה ראשונה → הערך 4.

ניתן להגדיר מערכים גם ממימדים גבוהים יותר:
![Alt text](https://predictivehacks.com/wp-content/uploads/2020/08/numpy_arrays.png)

NumPy מספקת גם פונקציות רבות ליצירת מערכים

- **`zeros(shape)`** – מחזירה מערך חדש בגודל המבוקש, שכל איבריו הם אפסים. שימושי לאתחול חישובים או שמירת מקום לנתונים עתידיים.  
- **`ones(shape)`** – יוצרת מערך חדש שכל איבריו הם אחדות. מתאים כבסיס לאתחול אחיד או למצבים בהם רוצים ערכי התחלה זהים.  
- **`full(shape, value)`** – מייצרת מערך שבו כל האיברים שווים לערך קבוע שניתן מראש.  
- **`eye(N)`** – מחזירה מטריצת יחידה ריבועית בגודל N×N, עם אחדות על האלכסון הראשי ואפסים בשאר המקומות. זהו אובייקט מרכזי באלגברה ליניארית.  
- **`random.random(shape)`** – יוצר מערך בגודל המבוקש שמלא במספרים אקראיים רציפים בין 0 ל־1. משמש בסימולציות, שיטות מונטה־קרלו ובדיקות סטטיסטיות.  

באמצעות פונקציות אלו ניתן לאתחל מערכים בצורה מהירה ויעילה בהתאם לצורך: ערכים קבועים, מטריצות יחידה או ערכים אקראיים:

In [10]:
import numpy as np

a = np.zeros((2,2))   # Create an array of all zeros
print(a)              

b = np.ones((1,2))    # Create an array of all ones
print(b)              

c = np.full((2,2), 7)  # Create a constant array
print(c)               

d = np.eye(2)         # Create a 2x2 identity matrix
print(d)              

e = np.random.random((2,2))  # Create an array filled with random values
print(e)                     


[[0. 0.]
 [0. 0.]]
[[1. 1.]]
[[7 7]
 [7 7]]
[[1. 0.]
 [0. 1.]]
[[0.92453552 0.30688484]
 [0.37444137 0.06485005]]


In [11]:
import json
from jupyterquiz import display_quiz
example = \
'''
   [{
    
        "question": "איזו פקודה ב־NumPy תיצור מטריצה בגודל 3x3 שכל האיברים בה הם 7?",
        "type": "many_choice",
        "answers": [
            {
                "answer": "np.array([[7]*3]*3)",
                "correct": false,
                "feedback": "נכון, אבל לא רק"
            },
            {
                "answer": "np.full((3,3), 7)",
                "correct": false,
                "feedback": "נכון, אבל לא רק"
            },
            {
                "answer": "np.ones((3,3)) * 7",
                "correct": false,
                "feedback": "נכון, אבל לא רק"

            },
            {
                "answer": "כל התשובות נכונות",
                "correct": true,
                "feedback": "נכון!"

            }
        ]
    }]
'''
myquiz = json.loads(example)
display_quiz(myquiz)

<IPython.core.display.Javascript object>

### Array indexing (אינדוקס במערכים)
ל־NumPy יש כמה דרכים לגשת לטווח אינדקסים במערכים.

#### Slicing (פריסה):
בדומה לרשומות בפייתון, גם מערכים ניתנים לפריסה (slice). מאחר שמערכים יכולים להיות רב־ממדיים, יש לציין פריסה לכל ממד.

ניצור מערך בגודל 3x4, כלומר שלוש שורות וארבעה עמודות:

In [12]:
# Create the following rank 2 array with shape (3, 4)
a = np.array([[1,2,3,4], 
              [5,6,7,8], 
              [9,10,11,12]])

#### חיתוך (Slicing) של מערך
נשתמש ב־slicing כדי לבחור תת־מערך:  
שתי השורות הראשונות (אינדקסים 0 ו־1) ועמודות 1 ו־2 (האינדקסים האמצעיים).

In [13]:
# Use slicing to pull out the subarray consisting of the first 2 rows
# and columns 1 and 2; b is the following array of shape (2, 2):
b = a[:2, 1:3]

#### קשר בין מערך ותת־מערך
חשוב לדעת: חיתוך במערכים של NumPy מחזיר **מבט (View)** אל הנתונים המקוריים, ולא עותק.  
לכן שינוי בתת־מערך ישפיע על המערך המקורי.

In [14]:
# A slice of an array is a view into the same data, so modifying it
# will modify the original array.
print(a[0, 1])   
b[0, 0] = 77     # b[0, 0] is the same piece of data as a[0, 1]
print(a[0, 1])   

2
77


In [15]:
import json
from jupyterquiz import display_quiz
example = \
'''
   [{
    
        "question": "בהינתן: a = np.array([1, 3, 5, 7, 9]) אילו פקודות יחזירו את כל האיברים הגדולים מ־4?",
        "type": "many_choice",
        "answers": [
            {
                "answer": "a[a > 4]",
                "correct": true,
                "feedback": "נכון"
            },
            {
                "answer": "a[4:]",
                "correct": false,
                "feedback": "לא נכון, זו גישה למערך החל מהאינדקס ה4"
            },
            {
                "answer": "a[a >= 5]",
                "correct": true,
                "feedback": "נכון"

            },
            {
                "answer": "a[> 4]",
                "correct": false,
                "feedback": "לא נכון, כתיבה לא חוקית"

            }
        ]
    }]
'''
myquiz = json.loads(example)
display_quiz(myquiz)

<IPython.core.display.Javascript object>

ניתן לשלב אינדוקס עם פריסה. שילוב כזה מפיק מערך בדרגה נמוכה יותר מהמקור.
#### אינדוקס של שורות: Rank 1 לעומת Rank 2
כאשר משלבים אינדוקס עם מספר שלם וחיתוך, מתקבלת **שורה בייצוג חד־ממדי (Rank 1)**.  
כאשר משתמשים רק בפריסה (slice), נשמרת **הדרגה המקורית (Rank 2)** ולכן מקבלים עדיין מערך דו־ממדי.

In [16]:
row_r1 = a[1, :]    # Rank 1 view of the second row of a
row_r2 = a[1:2, :]  # Rank 2 view of the second row of a
print(row_r1, row_r1.shape)  
print(row_r2, row_r2.shape)  

[5 6 7 8] (4,)
[[5 6 7 8]] (1, 4)


#### אינדוקס של עמודות: Rank 1 לעומת Rank 2
אותו עיקרון חל גם על עמודות:  
- אינדוקס עם מספר שלם יחזיר וקטור חד־ממדי.  
- שימוש בפריסה ישאיר את העמודה כתת־מערך דו־ממדי.


In [17]:
col_r1 = a[:, 1]
col_r2 = a[:, 1:2]
print(col_r1, col_r1.shape)  
print(col_r2, col_r2.shape)  

[77  6 10] (3,)
[[77]
 [ 6]
 [10]] (3, 1)


#### Integer array indexing (אינדוקס במערך באמצעות מספרים שלמים):
בעת פריסה מתקבל תמיד תת־מערך. אינדוקס בעזרת מערכי מספרים שלמים מאפשר לבנות מערך חדש מאיברים שרירותיים של מערך אחר.

ניצור מערך בגודל 3x2, כלומר שלוש שורות ושתי עמודות:

In [18]:
import numpy as np
a = np.array([[1,2],
              [3, 4],
              [5, 6]])
print(a)


[[1 2]
 [3 4]
 [5 6]]


ניתן לבחור איברים ממקומות שונים במערך בו־זמנית, באמצעות מתן רשימות של אינדקסים.  
התוצאה היא מערך חדש שבו מופיעים הערכים שנבחרו לפי המיקומים שצוינו.

In [19]:
print(a[[0, 1, 2],
        [0, 1, 0]]) 

[1 4 5]


התוצאה הקודמת זהה לבחירה ידנית של אותם איברים אחד־אחד, אך כאן עושים זאת בפקודה אחת.

In [20]:
print(np.array([a[0, 0], a[1, 1], a[2, 0]]))  

[1 4 5]


באמצעות אינדוקס שלם אפשר גם לבחור את אותו איבר מספר פעמים, ולבנות ממנו מערך חדש.

In [21]:
print(a[[0, 0], [1, 1]])  

[2 2]


גם כאן, אפשר לכתוב את אותו הדבר בעזרת גישה רגילה לכל איבר, ולבנות ידנית מערך חדש.

In [22]:
print(np.array([a[0, 1], a[0, 1]]))  

[2 2]


#### טריק נפוץ: לבחור/לעדכן איבר אחד מכל שורה באמצעות אינדקסים.
נבנה מערך בגודל 4x3 (ארבע שורות ושלושה עמודות):

In [23]:
a = np.array([[1,2,3], [4,5,6], [7,8,9], [10, 11, 12]])
print(a) 

[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]]


נגדיר מערך של אינדקסים (`b`) שאומר לנו איזה עמודה לבחור בכל שורה.  
במקרה הזה: בשורה הראשונה נבחר את העמודה ה־0, בשנייה את ה־2, בשלישית שוב את ה־0, וברביעית את ה־1.

In [24]:
b = np.array([0, 2, 0, 1])

השילוב `a[np.arange(4), b]` מאפשר לבחור איבר אחד מכל שורה:  
- `np.arange(4)` מייצר את הרשימה `[0,1,2,3]` – כלומר את כל השורות.  
- `b` נותן את האינדקס של העמודה המתאימה בכל שורה.  

ביחד מתקבלים ארבעה איברים – אחד מכל שורה.

In [25]:
print(a[np.arange(4), b])

[ 1  6  7 11]


ניתן גם לעדכן ישירות את אותם איברים שנבחרו, למשל להוסיף להם 10.  
כיוון שמדובר בגישה ישירה לנתוני המערך, השינוי מתבצע **בתוך המערך המקורי**.

In [26]:
a[np.arange(4), b] += 10
print(a)

[[11  2  3]
 [ 4  5 16]
 [17  8  9]
 [10 21 12]]


#### Boolean array indexing (אינדוקס בוליאני):
מאפשר לבחור איברים שמקיימים תנאי.

נבנה מערך בגודל 3x2:

In [27]:
import numpy as np

a = np.array([[1,2],
              [3,4],
              [5,6]])
print(a)

[[1 2]
 [3 4]
 [5 6]]


נשתמש במסיכה בוליאנית (Boolean Mask). נבדוק אילו איברים במערך גדולים מ־2.  
התוצאה היא מערך חדש באותו גודל, המכיל ערכי **True/False** לפי התנאי.

In [28]:
bool_idx = (a > 2)
print(bool_idx)

[[False False]
 [ True  True]
 [ True  True]]


כאשר משתמשים במערך הבוליאני כאינדקס, מתקבל מערך חד־ממדי עם כל האיברים עבורם התנאי מתקיים.

In [29]:
print(a[bool_idx])

[3 4 5 6]


ניתן לבצע את כל השלבים יחד בשורה אחת, על ידי הכנסת התנאי ישירות לאינדוקס.

In [30]:
print(a[a > 2])

[3 4 5 6]


### Datatypes (טיפוסי נתונים)
כל מערך ב־NumPy מכיל איברים מאותו טיפוס.
ספריית NumPy מספקת מגוון טיפוסים מספריים, לרוב היא תנחש את הטיפוס בעת יצירה, אך ניתן גם לציין טיפוס במפורש.

כאשר יוצרים מערך ממספרים שלמים רגילים, NumPy בוחר טיפוס שלם (int):

In [31]:
import numpy as np
x = np.array([1, 2])       # NumPy infers integer type
print(x.dtype)             # Usually int32 or int64 depending on the system

int64


כאשר יוצרים מערך ממספרים ממשיים (עם נקודה עשרונית), NumPy בוחר טיפוס נקודה צפה (float).  

In [32]:
x = np.array([1.0, 2.0])   # NumPy infers float type
print(x.dtype)             # Usually float64

float64


ניתן גם להכריח שימוש בטיפוס מסוים בזמן יצירת המערך, באמצעות הפרמטר `dtype`.  

In [33]:
x = np.array([1, 2], dtype=np.int64)   # Force explicit integer type (int64)
print(x.dtype)

int64


מעבר לטיפוסים בסיסיים כמו int ו־float, NumPy תומך גם בטיפוסים נוספים:  
- **מספרים מרוכבים** (`complex64`, `complex128`) – שימושי במכניקה קוונטית ובעיבוד אותות.  
- **טיפוסי נקודה צפה מדויקים יותר או פחות** (`float32`, `float64`) – בחירה בהתאם לאיזון בין דיוק למהירות/זיכרון.  
- **טיפוסי שלם שונים** (`int8`, `int16`, `int32`, `int64`) – שימושי כשיש מגבלות זיכרון או צורך ביעילות.  

יכולת השליטה בטיפוס הנתונים מאפשרת גם חיסכון במשאבים וגם התאמה לצרכים המדעיים הספציפיים של החישוב.

In [34]:
# Example with complex numbers
z = np.array([1+2j, 3+4j])    # Complex array
print(z.dtype)                # complex128 by default

# Example with lower precision floats
f = np.array([1.5, 2.5], dtype=np.float32)  # Explicit float32
print(f.dtype)

complex128
float32


### Array math (פעולות מתמטיות על מערכים)
ב־NumPy פונקציות מתמטיות בסיסיות פועלות **איבר־איבר** (elementwise).  
כלומר, הפעולה מתבצעת על כל זוג איברים מקבילים במערכים, והתוצאה היא מערך חדש.  
אותן פעולות זמינות גם כאופרטורים (`+`, `-`, `*`, `/`) וגם כפונקציות מתוך NumPy.

In [35]:
import numpy as np

x = np.array([[1,2],[3,4]], dtype=np.float64)
y = np.array([[5,6],[7,8]], dtype=np.float64)

# Elementwise sum; both produce the array
print(x + y)
print(np.add(x, y))

# Elementwise difference; both produce the array
print(x - y)
print(np.subtract(x, y))

# Elementwise product; both produce the array
print(x * y)
print(np.multiply(x, y))

# Elementwise division; both produce the array
print(x / y)
print(np.divide(x, y))

# Elementwise square root; produces the array
print(np.sqrt(x))


[[ 6.  8.]
 [10. 12.]]
[[ 6.  8.]
 [10. 12.]]
[[-4. -4.]
 [-4. -4.]]
[[-4. -4.]
 [-4. -4.]]
[[ 5. 12.]
 [21. 32.]]
[[ 5. 12.]
 [21. 32.]]
[[0.2        0.33333333]
 [0.42857143 0.5       ]]
[[0.2        0.33333333]
 [0.42857143 0.5       ]]
[[1.         1.41421356]
 [1.73205081 2.        ]]


In [36]:
import json
from jupyterquiz import display_quiz
example = \
'''
   [{
    
        "question": "מה ההבדל בין x * y ל־x.dot(y) ב־NumPy כאשר x ו־y הן מטריצות?",
        "type": "many_choice",
        "answers": [
            {
                "answer": "* עושה כפל מטריצות ו־dot כפל איבר־איבר",
                "correct": false,
                "feedback": "לא נכון"
            },
            {
                "answer": "* עושה כפל איבר־איבר ו־dot כפל מטריצות",
                "correct": true,
                "feedback": "נכון!"
            },
            {
                "answer": "שתיהן עושות כפל מטריצות",
                "correct": false,
                "feedback": "לא נכון"

            },
            {
                "answer": "שתיהן עושות כפל איבר־איבר",
                "correct": false,
                "feedback": "לא נכון"

            }
        ]
    }]
'''
myquiz = json.loads(example)
display_quiz(myquiz)

<IPython.core.display.Javascript object>

שימו לב: האופרטור `*` ב־NumPy מייצג **כפל איבר־איבר** (elementwise).  
כדי לבצע **כפל מטריציוני אמיתי** יש להשתמש בפונקציה `dot` — או כמתודה על המערך (`a.dot(b)`) או כפונקציה במודול (`np.dot(a,b)`).

In [37]:
import numpy as np

x = np.array([[1,2],[3,4]])
y = np.array([[5,6],[7,8]])

v = np.array([9,10])
w = np.array([11,12])

מכפלה פנימית (inner product) של שני וקטורים חד־ממדיים מחזירה סקלר.

In [38]:
print(v.dot(w))       
print(np.dot(v, w))   

219
219


מכפלת מטריצה־וקטור מחזירה וקטור חדש (Rank 1).

In [39]:
print(x.dot(v))       # [29 67]
print(np.dot(x, v))   # [29 67]

[29 67]
[29 67]


מכפלת מטריצה־מטריצה מחזירה מטריצה חדשה (Rank 2).

In [40]:
print(x.dot(y))
print(np.dot(x, y))

[[19 22]
 [43 50]]
[[19 22]
 [43 50]]


פונקציה שימושית נוספת היא sum:

In [41]:
import numpy as np

x = np.array([[1,2],[3,4]])

print(np.sum(x))  # Compute sum of all elements
print(np.sum(x, axis=0))  # Compute sum of each column
print(np.sum(x, axis=1))  # Compute sum of each row


10
[4 6]
[3 7]


לעיתים נרצה לשנות צורה או לבצע מניפולציות אחרות על המערך. הדוגמה הפשוטה ביותר היא טרנספוזיציה (T):

In [42]:
import numpy as np

x = np.array([[1,2], [3,4]])
print(x)    
print(x.T)  

# Note that taking the transpose of a rank 1 array does nothing:
v = np.array([1,2,3])
print(v)    
print(v.T)  


[[1 2]
 [3 4]]
[[1 3]
 [2 4]]
[1 2 3]
[1 2 3]


### Broadcasting (שידור)
Broadcasting הוא מנגנון שמאפשר ל־NumPy לבצע פעולות אריתמטיות בין מערכים בצורות שונות, ע״י שימוש חוזר במערך קטן יותר מול מערך גדול יותר.

ללא Broadcasting , אם נרצה להוסיף וקטור קבוע לכל שורה במטריצה, נאלץ להקצות מערך בגודל הסופי ולהשתמש בלולאה (יקר חישובית) על מנת להוסיף את הוקטור לכל שורה במטריצה:

In [43]:
import numpy as np

# We will add the vector v to each row of the matrix x,
# storing the result in the matrix y
x = np.array([[1,2,3], [4,5,6], [7,8,9], [10, 11, 12]])
v = np.array([1, 0, 1])
y = np.empty_like(x)   # Create an empty matrix with the same shape as x

# Add the vector v to each row of the matrix x with an explicit loop
for i in range(4):
    y[i, :] = x[i, :] + v

print(y)


[[ 2  2  4]
 [ 5  5  7]
 [ 8  8 10]
 [11 11 13]]


Broadcasting חוסך את הצורך ביצירת עותקים ופשוט כותבים:

In [44]:
import numpy as np

# We will add the vector v to each row of the matrix x,
# storing the result in the matrix y
x = np.array([[1,2,3], [4,5,6], [7,8,9], [10, 11, 12]])
v = np.array([1, 0, 1])
y = x + v  # Add v to each row of x using broadcasting
print(y) 


[[ 2  2  4]
 [ 5  5  7]
 [ 8  8 10]
 [11 11 13]]


למה זה עובד? כי NumPy “מרחיבה” את v לצורה (4,3) באופן לוגי (ללא הקצאת זיכרון מיותרת).

מצורפת תמונה להמחשה של broadcasting:

![Alt text](https://jakevdp.github.io/PythonDataScienceHandbook/figures/02.05-broadcasting.png)


#### כללי ה־broadcasting:

כדי שהפעולה תצליח, חייבים להתקיים הכללים הבאים:

1. השוואת דרגות (rank) המערכים

אם למערכים יש מספר ממדים שונה, מוסיפים 1 בתחילת ה־shape של המערך הקטן יותר, עד שאורכי הצורות שווים.
למשל:

(4, 3)     ← מערך דו־ממדי
(4)        ← מערך חד־ממדי

⇒  (4, 3)
    (4, 1) ← נוספה 1 בתחילת ה־shape

2. תאימות ממדית
בממד מסוים שתי הצורות נחשבות תואמות אם: הגודל בממד הזה שווה בשני המערכים, או לאחד מהם יש גודל 1 בממד הזה.

דוגמה (תואמים):
(3, 1, 5)
(3, 4, 5)   ← בממד האמצעי: 1 מול 4 → תואם

דוגמה (לא תואמים):
(3, 2)
(3, 4)   ← בממד הראשון: 2 מול 4 → לא תואם

3. בדיקת כל הממדים

Broadcasting אפשרי רק אם יש תאימות בכל הממדים לפי סעיף 2.

4. קביעת הצורה הסופית

הצורה של התוצאה היא ה־shape המתקבל על ידי לקיחת המקסימום רכיב־לרכיב בין שני ה־shape-ים.
לדוגמה: 
(3, 1, 5)
(1, 4, 5)
⇒ (3, 4, 5)

5. שכפול לוגי של ממדים עם 1

בממד שבו למערך אחד יש גודל 1 ולשני יש גודל גדול יותר, המערך עם ה־1 “משוכפל” לאורך הממד הזה כך שיתאים לגודל השני — לוגית בלבד, ללא הקצאת זיכרון נוספת.
לדוגמא:
מערך A בגודל (2, 3) ומערך B בגודל (3). NumPy הופכת את B ל־(1, 3), משכפלת אותו על פני השורה (ממד 0), ומבצעת את הפעולה איבר־איבר לקבלת מערך בגודל (2, 3).

[להסבר נוסף](https://scipy.github.io/old-wiki/pages/EricsBroadcastingDoc)

### דוגמאות נוספות:
#### כפל חיצוני (Outer Product) באמצעות Broadcasting
כדי לחשב כפל חיצוני בין שני וקטורים, נרצה להפוך את הווקטור הראשון לעמודה ולהכפיל איבר־איבר בווקטור השני. התוצאה היא מטריצה שכל איבר בה הוא מכפלת איבר מהווקטור הראשון באיבר מתאים מהווקטור השני.


In [45]:
import numpy as np

v = np.array([1, 2, 3])    # shape (3,)
w = np.array([4, 5])       # shape (2,)

# Make v a column vector, then broadcast against w
outer_vw = np.reshape(v, (3, 1)) * w   # (3,1) * (2,) -> (3,2)
print(outer_vw)

# Alternative, more concise forms:
outer_vw2 = v[:, None] * w             # (3,1) * (2,) -> (3,2)
print(outer_vw2)

# Using dedicated function for clarity:
outer_vw3 = np.outer(v, w)             # outer product directly
print(outer_vw3)


[[ 4  5]
 [ 8 10]
 [12 15]]
[[ 4  5]
 [ 8 10]
 [12 15]]
[[ 4  5]
 [ 8 10]
 [12 15]]


#### הוספת וקטור לכל שורה (Row-wise)
כאשר ממד הווקטור מתאים למספר העמודות במטריצה, ניתן להוסיף אותו לכל שורה באמצעות Broadcasting.


In [46]:
x = np.array([[1, 2, 3],
              [4, 5, 6]])              # shape (2,3)

row_added = x + v                      # (2,3) + (3,) -> (2,3)
print(row_added)

# Explicit broadcasting with an added row axis:
row_added2 = x + v[None, :]            # (2,3) + (1,3) -> (2,3)
print(row_added2)

[[2 4 6]
 [5 7 9]]
[[2 4 6]
 [5 7 9]]


#### הוספת וקטור לכל עמודה (Column-wise)
ניתן להוסיף וקטור באורך מספר השורות לכל עמודה. אפשר לעשות זאת ע"י טרנספוזיציה או ע"י שינוי צורה מפורש של הווקטור לעמודה.

In [47]:
# Using transpose trick
col_added = (x.T + w).T                # (3,2)->(3,2), then back to (2,3)
print(col_added)

# More direct and readable:
col_added2 = x + w[:, None]            # (2,3) + (2,1) -> (2,3)
print(col_added2)


[[ 5  6  7]
 [ 9 10 11]]
[[ 5  6  7]
 [ 9 10 11]]


#### כפל בסקלר (Scalar) באמצעות Broadcasting
סקלר מתנהג כמו מערך ריק ממדים ומתרחב לצורת המטריצה.


In [48]:
scaled = x * 2
print(scaled)

scaled_div = x / 2
print(scaled_div)

shifted = x + 3
print(shifted)

[[ 2  4  6]
 [ 8 10 12]]
[[0.5 1.  1.5]
 [2.  2.5 3. ]]
[[4 5 6]
 [7 8 9]]


#### דוגמאות נוספות של Broadcasting שימושי

1) הכפלת כל עמודה במשקל שונה (למשל לנרמול).  

In [49]:
# 1) Column-wise scaling
weights = np.array([1.0, 10.0, 100.0])  # shape (3,)
scaled_cols = x * weights                # (2,3) * (3,) -> (2,3)
print(scaled_cols)

[[  1.  20. 300.]
 [  4.  50. 600.]]


2) החסרת ממוצע עמודות (Centering) — פעולת הכנה נפוצה לניתוח נתונים.  

In [50]:
# 2) Centering by column means
col_means = x.mean(axis=0, keepdims=True)  # shape (1,3)
x_centered = x - col_means                  # (2,3) - (1,3) -> (2,3)
print(x_centered)

[[-1.5 -1.5 -1.5]
 [ 1.5  1.5  1.5]]


3) נרמול שורות לאורך יחידה — שימושי בווקטורים/דגימות.  

In [51]:
# 3) Row normalization to unit length
row_norms = np.linalg.norm(x, axis=1, keepdims=True)  # shape (2,1)
x_row_unit = x / row_norms                             # (2,3)/(2,1) -> (2,3)
print(x_row_unit)

[[0.26726124 0.53452248 0.80178373]
 [0.45584231 0.56980288 0.68376346]]


4) כפל חיצוני בין וקטורים באורכים אחרים להמחשת הכללה.

In [52]:
# 4) Another outer product example with different sizes
p = np.array([2, 0, -1, 4])   # shape (4,)
q = np.array([-3, 5, 2])      # shape (3,)
outer_pq = p[:, None] * q     # (4,1) * (3,) -> (4,3)
print(outer_pq)

[[ -6  10   4]
 [  0   0   0]
 [  3  -5  -2]
 [-12  20   8]]


בדרך כלל broadcasting הופך את הקוד לקצר ומהיר יותר — מומלץ להשתמש בו כשאפשר.

In [53]:
import json
from jupyterquiz import display_quiz

example = \
'''
[{
  "question": "מה ידפיס קטע הקוד הבא?<br><br>a = np.ones((2, 1, 5))<br><br>b = np.ones((1, 4, 5))<br><br>print((a+b).shape)",
  "type": "many_choice",
  "answers": [
    {"answer": "(2, 4, 5)", "correct": true,  "feedback": "נכון!"},
    {"answer": "(2, 1, 5)", "correct": false, "feedback": "לא נכון"},
    {"answer": "(1, 4, 5)", "correct": false, "feedback": "לא נכון"},
    {"answer": "לא ניתן לבצע broadcasting", "correct": false, "feedback": "לא נכון"}
  ]
}]
'''
myquiz = json.loads(example)
display_quiz(myquiz)


<IPython.core.display.Javascript object>